In [2]:
import numpy as np
import pandas as pd
import itertools
import random
import sklearn
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import math
from collections import Counter
import altair as alt
from IPython.display import display
#work with huge nb of rows on altair
alt.data_transformers.disable_max_rows()
import seaborn as sns
import datetime
from functools import wraps
from time import time

### Load data

In [3]:
######## TO MODIFY ###########
path_to_transactions_df = '../data/uniqlo_parquet_files_consolidated_transformed_transaction_lines.parquet'
path_to_inventory_uniformity_df = '../data/uniqlo_metrics_inventory_uniformity_product_2021-10-05.parquet'
#############################

transactions_df = pd.read_parquet(path_to_transactions_df,engine="pyarrow")
inventory_uniformity_df = pd.read_parquet(path_to_inventory_uniformity_df,engine="pyarrow")


### Preprocess data 

In [4]:
transactions_df = transactions_df[(~transactions_df.customer_id.isna()) &
                                  (transactions_df.customer_id != 'nan')].copy()

### Number of customer per number of transaction

In [5]:
def plot_nb_customer_per_nb_transaction(graph="line",add_quartile=True):
    nb_customer_per_nb_transaction = transactions_df["customer_id"].value_counts()
    counter = Counter(nb_customer_per_nb_transaction.tolist()).most_common()
    transaction,customer = [x[0] for x in counter],[x[1] for x in counter]
    source = pd.DataFrame({
      'nb_transaction': transaction,
      'nb_customer': customer
    })
    
    if graph=="line":
        plt = alt.Chart(source).interactive(bind_x=True, bind_y=False).mark_line(point=True).encode(
            x=alt.X('nb_transaction',scale=alt.Scale(domain=[0, 100])),
            y='nb_customer'
        ).properties(
            title='Number of customer per number of transaction'
        )
    elif graph=="bar":
        plt = alt.Chart(source).interactive(bind_x=True, bind_y=False).mark_bar().encode(
            x= alt.X("nb_transaction", bin=alt.Bin(maxbins=10000),scale=alt.Scale(domain=[0, 100])),
            y='nb_customer',
        ).properties(
            title='Number of customer per number of transaction'
        )
    if add_quartile:
        print("quartile 25/50/75 : "  + str(np.percentile(nb_customer_per_nb_transaction.tolist(), [25, 50, 75])))
        quartile  =  pd.DataFrame({"quartile" : np.percentile(nb_customer_per_nb_transaction.tolist(), [25, 50, 75])})
        Q1 = (
            alt.Chart(quartile)
            .mark_rule()
            .encode(
                x = "quartile"
            )
        )
        display(plt + Q1)
    else :
        display(plt)

#### The plots are interactive : 

For the line chart and the bar chart, you can **zoom** in and out and **click-n'-drag** the plot to translate on the x-axis.

In [6]:
plot_nb_customer_per_nb_transaction(graph="line")
plot_nb_customer_per_nb_transaction(graph="bar")

quartile 25/50/75 : [2. 3. 6.]


alt.LayerChart(...)

quartile 25/50/75 : [2. 3. 6.]


alt.LayerChart(...)

### Distribution of Item quantity

In [3]:
def plot_item_quantity_distribution(graph="line",add_quartile=True):
    quatity_per_item_per_customer = transactions_df.groupby(["customer_id","variant_id"]).agg({'quantity': 'sum'})["quantity"]
    counter = Counter(quatity_per_item_per_customer.tolist()).most_common()
    quantity,count = [x[0] for x in counter],[x[1] for x in counter]
    source = pd.DataFrame({
      'item_quantity': quantity,
      'counter': count
    })
    if graph=="line":
        plt = alt.Chart(source).interactive(bind_x=True, bind_y=False).mark_line(point=True).encode(
            x=alt.X('item_quantity',scale=alt.Scale(domain=[0, 100])),
            y='counter'
        ).properties(
            title='Distribution of item quantity bought by customer'
        )
    elif graph=="bar":
        plt = alt.Chart(source).interactive(bind_x=True, bind_y=False).mark_bar().encode(
            x= alt.X("item_quantity", bin=alt.Bin(maxbins=1000),scale=alt.Scale(domain=[0, 100])),
            y='counter',
        ).properties(
            title='Distribution of item quantity bought by customer'
        )
    if add_quartile:
        print("quartile 25/50/75 : "  + str(np.percentile(quatity_per_item_per_customer.tolist(), [25, 50, 75])))
        quartile  =  pd.DataFrame({"quartile" : np.percentile(quatity_per_item_per_customer.tolist(), [25, 50, 75])})
        Q1 = (
            alt.Chart(quartile)
            .mark_rule()
            .encode(
                x = "quartile"
            )
        )
        display(plt + Q1)
    else :
        display(plt)

In [4]:
plot_item_quantity_distribution("line")
plot_item_quantity_distribution("bar")

NameError: name 'transactions_df' is not defined

### Distribution of the mean selling_price 

In [135]:
def plot_mean_selling_price_distribution(graph="line",add_quartile=True):
    mean_selling_price_distribution = transactions_df.groupby(["customer_id"]).agg({'selling_price': 'mean'})["selling_price"]
    counter = Counter(mean_selling_price_distribution.tolist()).most_common()
    mean_selling_price,customer = [x[0] for x in counter],[x[1] for x in counter]
    source = pd.DataFrame({
      'mean_selling_price': mean_selling_price,
      'nb_customer': customer
    })
    if graph=="line":
        plt = alt.Chart(source).interactive(bind_x=True, bind_y=False).mark_line(point=True).encode(
            x=alt.X('mean_selling_price',scale=alt.Scale(domain=[0, 100])),
            y='nb_customer'
        ).properties(
            title='Distribution of the mean selling price '
        )
    elif graph=="bar":
        plt = alt.Chart(source).interactive(bind_x=True, bind_y=False).mark_bar().encode(
            x= alt.X("mean_selling_price", bin=alt.Bin(maxbins=50),scale=alt.Scale(domain=[0, 100])),
            y='nb_customer',
        ).properties(
            title='Distribution of the mean selling price '
        )
    if add_quartile:
        print("quartile 25/50/75 : "  + str(np.percentile(mean_selling_price_distribution.tolist(), [25, 50, 75])))
        quartile  =  pd.DataFrame({"quartile" : np.percentile(mean_selling_price_distribution.tolist(), [25, 50, 75])})
        Q1 = (
            alt.Chart(quartile)
            .mark_rule()
            .encode(
                x = "quartile"
            )
        )
        display(plt + Q1)
    else :
        display(plt)

In [136]:
plot_mean_selling_price_distribution("bar")

quartile 25/50/75 : [13.65       18.01111111 24.9       ]


alt.LayerChart(...)

### Number of selling per item

In [155]:
def plot_selling_per_item(graph="line",add_quartile=True):
    selling_per_item = transactions_df.groupby("variant_id").agg({"quantity":"sum"})["quantity"]
    counter = Counter(selling_per_item.tolist()).most_common()
    selling, nb_item = [x[0] for x in counter],[x[1] for x in counter]
    source = pd.DataFrame({
      'item_sold': selling,
      'nb_item': nb_item
    })
    if graph=="line":
        plt = alt.Chart(source).interactive(bind_x=True, bind_y=False).mark_line(point=True).encode(
            x=alt.X('item_sold',scale=alt.Scale(domain=[0, 100])),
            y='nb_item'
        ).properties(
            title='Number of selling per item'
        )
    elif graph=="bar":
        plt = alt.Chart(source).interactive(bind_x=True, bind_y=False).mark_bar().encode(
            x= alt.X("item_sold", bin=alt.Bin(maxbins=1000),scale=alt.Scale(domain=[0, 2000])),
            y='nb_item',
        ).properties(
            title='Number of selling per item'
        )
    if add_quartile:
        print("quartile 25/50/75 : "  + str(np.percentile(selling_per_item.tolist(), [25, 50, 75])))
        quartile  =  pd.DataFrame({"quartile" : np.percentile(selling_per_item.tolist(), [25, 50, 75])})
        Q1 = (
            alt.Chart(quartile)
            .mark_rule()
            .encode(
                x = "quartile"
            )
        )
        display(plt + Q1)
    else :
        display(plt)

In [156]:
plot_selling_per_item("line")
plot_selling_per_item("bar")

quartile 25/50/75 : [  81.    432.   1280.25]


alt.LayerChart(...)

quartile 25/50/75 : [  81.    432.   1280.25]


alt.LayerChart(...)

# Actual engine 

In [212]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        arg = args[0] if len(args)>=1 else "" 
        print('func:%r  took: %2.4f sec' % \
          (f.__name__, te-ts))
        return result
    return wrap

def merge_customer_recommendation_dfs(transactions_df, inventory_uniformity_df, recommendation_level):
    # remove row with nan or missing values in customer_id column
    transactions_df = transactions_df[(~transactions_df.customer_id.isna()) &
                                      (transactions_df.customer_id != 'nan')].copy()
    # recommendation based on all products
    recommendation_df1 = create_customer_recommendation_df(transactions_df, recommendation_level)

    variant_ids = inventory_uniformity_df[inventory_uniformity_df.is_uniformly_distributed].variant_id.unique().tolist()
    transactions_df = transactions_df[transactions_df.variant_id.isin(variant_ids)].copy()
    
    # recommendation based on only in stock products
    recommendation_df2 = create_customer_recommendation_df(transactions_df, recommendation_level)
    return pd.merge(recommendation_df1, recommendation_df2, on='customer_id', how='left', suffixes=['', '_instock'])

def create_customer_recommendation_df(transactions_df, recommendation_level):
    customer_product_df = transactions_df.groupby(['customer_id', recommendation_level])\
        .agg({'quantity': 'sum'}).reset_index()
    utility_matrix, correlation_matrix = create_correlation_matrix(customer_product_df, recommendation_level)
    
    # get the top products (based on the number of items bought) for each customer 
    idx = customer_product_df.groupby(['customer_id'])['quantity'].transform(max) == customer_product_df['quantity']
    customer_top_product_df = customer_product_df[idx]
    
    # get the second top products (based on the number of items bought) for each customer 
    customer_second_product_df = customer_product_df[~idx]
    idx2 = customer_second_product_df.groupby(['customer_id'])['quantity'].transform(max) == customer_second_product_df['quantity']
    customer_second_product_df = customer_second_product_df[idx2]
    
    top_product_recommendation = create_customer_product_recommendation(customer_top_product_df, recommendation_level,
                                                                        utility_matrix, correlation_matrix)
    
    second_product_recommendation = create_customer_product_recommendation(customer_second_product_df,
                                                                           recommendation_level, utility_matrix,
                                                                           correlation_matrix)
    
    
    recommendation_df = pd.merge(top_product_recommendation, second_product_recommendation, on='customer_id',
                                 how='left', suffixes=['', '_second'])
    # fill na value to [""] because some customer do not have second product recommendation
    recommendation_df['recommended_products_second'] = recommendation_df['recommended_products_second'].fillna("").apply(list)
    
    # concat list from top and second product. However we can still have a product recommended twice.
    recommendation_df[f'recommended_{recommendation_level}'] = recommendation_df['recommended_products'].apply(
        lambda x: list(set(x))) + recommendation_df['recommended_products_second'].apply(lambda x: list(set(x)))

    return recommendation_df[['customer_id', 'variant_id', f'recommended_{recommendation_level}']]

def create_customer_product_recommendation(customer_top_product_df, recommendation_level, utility_matrix,
                                           correlation_matrix):
    # keep only one item 
    customer_top_product_df = customer_top_product_df.drop_duplicates(subset='customer_id', keep="last")
    product_id_df = pd.DataFrame(customer_top_product_df[recommendation_level].unique(), columns=[recommendation_level])
    product_id_df['recommended_products'] = product_id_df[recommendation_level].apply(
        lambda x: list_similar_products(x, utility_matrix, correlation_matrix))

    customer_product_df = customer_top_product_df.merge(product_id_df, on=recommendation_level)
    return customer_product_df

def create_correlation_matrix(customer_product_df, recommendation_level):
    ratings_utility_matrix = customer_product_df.pivot_table(values='quantity', index='customer_id',
                                                             columns=recommendation_level, fill_value=0)
    utility_matrix = ratings_utility_matrix.T
    SVD = TruncatedSVD(n_components=3)
    decomposed_matrix = SVD.fit_transform(utility_matrix)
    correlation_matrix = np.corrcoef(decomposed_matrix)
    return utility_matrix, correlation_matrix

def list_similar_products(product_id, utility_matrix, correlation_matrix):
    product_names = list(utility_matrix.index)
    product_id_index = product_names.index(product_id)
    correlation_product_id = correlation_matrix[product_id_index]
    recommendation = list(utility_matrix.index[correlation_product_id > 0.95])
    return recommendation[0:15]

def _create_metrics():
    transactions_df = pd.read_parquet('uniqlo_parquet_files_consolidated_transformed_transaction_lines.parquet',engine="pyarrow")
    inventory_uniformity_df = pd.read_parquet('uniqlo_metrics_inventory_uniformity_product_2021-10-05.parquet',engine="pyarrow")
    transactions_date_df = transactions_df[transactions_df["order_date"] > datetime.datetime(2021,9,25)]
    customer_df = merge_customer_recommendation_dfs(transactions_date_df, inventory_uniformity_df, "variant_id")
    print(customer_df)
def main():
    _create_metrics()

In [214]:
main()

0.9969911575317383
1.64335298538208
1.6673719882965088
0.3196370601654053
0.5422122478485107
0.5460860729217529
               customer_id variant_id  \
0                    09167     440254   
1     US051406141454460474     440254   
2        US621111112833259     440254   
3        US621111113567410     440254   
4        US621111116381070     440254   
...                    ...        ...   
6153     US621111116385295     442997   
6154     US621111116385382     441751   
6155     US621111116385560     428857   
6156     US621111116385905     428856   
6157     US621111116386054     439343   

                                 recommended_variant_id variant_id_instock  \
0     [424600, 419503, 419993, 424941, 422922, 42236...             440254   
1     [424600, 419503, 419993, 424941, 422922, 42236...             440254   
2     [424600, 419503, 419993, 424941, 422922, 42236...             440254   
3     [424600, 419503, 419993, 424941, 422922, 42236...             440254   
4    